# Generating protein-DNA complexes using web3DNA

This notebook automates the generation of mutant Myc/Max–DNA complexes by programmatically driving the w3DNA “mutation” web tool via a headless Chrome browser. It takes as inputs (1) a reference PDB (`MycMax_PDB.pdb` (processed PDB, original is PDB entry `1NKP`), containing DNA chains A/B and protein chains E/F, waters removed, and histidine protonation pre-assigned) and (2) a CSV file (`dataset.csv`) listing 36-bp DNA target sequences. For each sequence, the script uploads the PDB, selects the appropriate base-substitution radio buttons for both strands, submits the mutation job, and then downloads the resulting PDB file (named `MycMax_PDB_<sequence>.pdb`). All outputs are saved alongside the original PDB.

_Note: This script was run locally, not on HPC_

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException  
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import urllib
import os
import urllib.request
import pandas as pd

## Read inputs and set residues to mutate

In [2]:
# Note: the input PDB is DNA chains A,B and protein chains E,F from 1NKP with waters removed.
# The histidine protonation states have already been assigned using pdb2pqr websever.

path_pdb = '../DNA_library/MycMax_PDB.pdb' #Input reference PDB (output will be in the same directory)
path_dataset = 'dataset.csv' #Input library of DNA sequences to model

resid_mutate=[i for i in range(1,36+1)] # Residues to mutate (here I'm doing the whole 36-bp DNA)
resid_paired_mutate = [i for i in range(72,37-1,-1)] # Basepairing Residues

## Perform mutations

In [3]:
dataset = pd.read_csv(path_dataset)

seqs = dataset['SEQUENCE'].to_list() #Sequences I wish to mutate to

#'Base':[button to click on webpage, paired base]
bp_dict = {'A':[0,'T'],
           'C':[1,'G'],
           'G':[2,'C'],
           'T':[3,'A']}

In [4]:
# Set up the Chrome driver
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--disable-gpu')

In [ ]:
max_retries = 10  # Define the maximum number of retries
driver = webdriver.Chrome(service=service, options=options)

for seq in seqs:
    retry_count = 0
    success = False
    
    while retry_count < max_retries and not success:
        try:
            # Step 1: Access the webpage
            driver.get("http://web.x3dna.org/index.php/mutation")
            
            # Step 2: Upload PDB file
            search_box = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, "userfile"))
            )
            search_box.send_keys(path_pdb)
            
            # Step 3: Go to base mutation page
            submit_button = WebDriverWait(driver, 20).until(
                EC.element_to_be_clickable((By.XPATH, "//form[@action='http://web.x3dna.org/mutation_file/upload']//input[@type='submit']"))
            )
            submit_button.click()

            # Step 4: Mutate Residues
            for i, res in enumerate(list(seq)):
                # Mutate residue
                resid = resid_mutate[i]
                value = bp_dict[res][0]
                
                mutation_radio_button = WebDriverWait(driver, 30).until(
                    EC.element_to_be_clickable((By.XPATH, f"//td[@id='r{resid}']//input[@value={value}]"))
                )
                mutation_radio_button.click()
                
                # Mutate basepair
                resid_bp = resid_paired_mutate[i]
                value_bp = bp_dict[bp_dict[res][1]][0]
                
                mutation_radio_button = WebDriverWait(driver, 30).until(
                    EC.element_to_be_clickable((By.XPATH, f"//td[@id='r{resid_bp}']//input[@value={value_bp}]"))
                )
                mutation_radio_button.click()

            # Step 5: Continue to Next Page
            submit_mutant = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.XPATH, "//input[@type='Submit']"))
            )
            submit_mutant.click()

            # Step 6: Download Mutant
            download_link = WebDriverWait(driver, 60).until(
                EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'mutate/main_view02.pdb')]"))
            )
            file_url = download_link.get_attribute('href')
            
            new_filename = path_pdb.replace('.pdb', f'_{seq}.pdb')

            # Download the file
            urllib.request.urlretrieve(file_url, new_filename)
            print(f"Mutated PDB file downloaded and saved as {new_filename}")
            
            success = True  # Mark as successful if everything works

        except TimeoutException:
            retry_count += 1
            print(f"Timeout occurred for sequence {seq}. Retrying... ({retry_count}/{max_retries})")
            if retry_count == max_retries:
                print(f"Failed to process sequence {seq} after {max_retries} attempts.")
        
        except Exception as e:
            print(f"An error occurred: {e}")
            break  # Optionally break the loop if a critical error occurs

# Quit the driver after the loop finishes
driver.quit()
